# Processing and analyzing weblog data

# Importing libraries

In [1]:
import pandas as pd
import sys
import numpy as np
import pandas_profiling
from datetime import timedelta

# Processing and loading the data

In [2]:
filePath= r'D:\WeblogChallenge-master\data\2015_07_22_mktplace_shop_web_log_sample.log'

convertRequest= lambda x: 'NULL' if x == "- - - " else x
convertUser= lambda x: 'NULL' if x=='-' else x
convertCliper= lambda x: 'NULL' if x=='-' else x
convertProtocol= lambda x: 'NULL' if x=='-' else x
convertBackendPort= lambda x: 'NULL' if x=='-' else x

df= pd.read_csv (filePath, converters= { 'request': convertRequest, 'user_agent':convertUser, 'backend:port': convertBackendPort,'ssl_cipher':convertCliper, 'ssl_protocol': convertProtocol},
               delim_whitespace= True, quotechar= '"', 
                 names=['timestamp', 'elb', 'client:port', 'backend:port', 'request_processing_time','backend_processing_time', 
                       'response_processing_time', 'elb_status_code','backend_status_code', 'received_bytes', 'sent_bytes', 'request',
                       'user_agent', 'ssl_cipher', 'ssl_protocol'])

# Looking into the data

In [3]:
#pandas_profiling.ProfileReport(df)

# Sessionizing the data

In [4]:
# Sort the data and find the previous time for each client
dfIpReqTime= df[['client:port', 'request', 'timestamp']].sort_values(['client:port','timestamp'], ascending= [True, True])
dfIpReqTime['prev_timestamp']= dfIpReqTime.groupby("client:port") ['timestamp'].shift(1)
dfIpReqTime.head()

,client:port,request,timestamp,prev_timestamp
109956,1.186.101.79:50613,GET https://paytm.com:443/shop/wallet/txnhisto...,2015-07-22T10:45:55.885488Z,NaN
122788,1.186.101.79:50613,GET https://paytm.com:443/shop/v1/frequentorde...,2015-07-22T10:46:27.839734Z,2015-07-22T10:45:55.885488Z
134527,1.186.101.79:50613,POST https://paytm.com:443/papi/v1/expresscart...,2015-07-22T10:46:56.591943Z,2015-07-22T10:46:27.839734Z
136716,1.186.101.79:50613,POST https://paytm.com:443/api/v1/expresscart/...,2015-07-22T10:47:01.782695Z,2015-07-22T10:46:56.591943Z
138619,1.186.101.79:50613,GET https://paytm.com:443/shop/summary/1116587...,2015-07-22T10:47:06.893987Z,2015-07-22T10:47:01.782695Z


## Window time: 15 minutes

In [5]:
# Define T as a session window of 15 minutes
T = timedelta(seconds=15*60)

dfIpReqTime['timestamp']= pd.to_datetime(dfIpReqTime['timestamp'])
dfIpReqTime['prev_timestamp']= pd.to_datetime(dfIpReqTime['prev_timestamp'])
dfIpReqTime['interval'] = (dfIpReqTime['timestamp'] - dfIpReqTime['prev_timestamp'])
dfIpReqTime['new_session'] = ((dfIpReqTime['timestamp'] - dfIpReqTime['prev_timestamp'])>=T).astype(int)
dfIpReqTime.head()


,client:port,request,timestamp,prev_timestamp,interval,new_session
109956,1.186.101.79:50613,GET https://paytm.com:443/shop/wallet/txnhisto...,2015-07-22 10:45:55.885488,NaT,NaT,0
122788,1.186.101.79:50613,GET https://paytm.com:443/shop/v1/frequentorde...,2015-07-22 10:46:27.839734,2015-07-22 10:45:55.885488,00:00:31.954246,0
134527,1.186.101.79:50613,POST https://paytm.com:443/papi/v1/expresscart...,2015-07-22 10:46:56.591943,2015-07-22 10:46:27.839734,00:00:28.752209,0
136716,1.186.101.79:50613,POST https://paytm.com:443/api/v1/expresscart/...,2015-07-22 10:47:01.782695,2015-07-22 10:46:56.591943,00:00:05.190752,0
138619,1.186.101.79:50613,GET https://paytm.com:443/shop/summary/1116587...,2015-07-22 10:47:06.893987,2015-07-22 10:47:01.782695,00:00:05.111292,0


In [6]:
# create the session_id
dfIpReqTime['increment'] = dfIpReqTime.groupby("client:port")['new_session'].cumsum()
dfIpReqTime['session_id'] = dfIpReqTime['client:port'].astype(str) + '_' + dfIpReqTime['increment'].astype(str)

# Determine the average session time

In [7]:
# Create a summary table
dfSessionDuration= pd.DataFrame(dfIpReqTime[['session_id','interval']])

dfSessionDuration.drop_duplicates(inplace=True)
dfSessionDuration.head()

,session_id,interval
109956,1.186.101.79:50613_0,NaT
122788,1.186.101.79:50613_0,00:00:31.954246
134527,1.186.101.79:50613_0,00:00:28.752209
136716,1.186.101.79:50613_0,00:00:05.190752
138619,1.186.101.79:50613_0,00:00:05.111292


In [8]:
# Calculate average session time
count= pd.value_counts( pd.value_counts(dfSessionDuration.session_id.unique()))
sumAll= dfSessionDuration.interval.sum()

totalNs=sumAll / np.timedelta64(1, 'ns')

avg1= totalNs/count

print("Average session time is {}".format(list(pd.to_timedelta(avg1))))



Average session time is [Timedelta('0 days 00:25:52.742856')]


# Determine unique URL visits per session

In [10]:
np.set_printoptions(threshold=sys.maxsize, linewidth= np.inf)

dfIpReqTime.groupby(['session_id'])['request'].apply(lambda x: list(set(x.unique().tolist())))

session_id
1.186.101.79:50613_0      [POST https://paytm.com:443/api/v1/expresscart...
1.186.101.79:50614_0      [GET https://paytm.com:443/shop/wallet/balance...
1.186.101.79:50895_0      [GET https://paytm.com:443/shop/wallet/txnhist...
1.186.101.79:50959_0      [GET https://paytm.com:443/shop/wallet/balance...
1.186.101.79:50968_0      [GET https://paytm.com:443/paytmwallet HTTP/1....
1.186.101.79:50991_0      [GET https://paytm.com:443/shop/cart?channel=w...
1.186.101.79:50999_0      [GET https://paytm.com:443/shop/wallet/balance...
1.186.103.240:65129_0     [GET https://paytm.com:443/shop/summary/111804...
1.186.103.78:25143_0      [GET https://paytm.com:443/favicon.ico HTTP/1....
1.186.108.213:62035_0     [GET https://paytm.com:443/shop/cart?channel=w...
1.186.108.213:62110_0     [GET https://paytm.com:443/shop/user/address?c...
1.186.108.230:53166_0     [GET https://paytm.com:443/shop/summary/undefi...
1.186.108.242:50347_0     [GET https://paytm.com:443/shop/v1/frequentord...
1

# Find the most engaged users

## Using the session duration

In [11]:
# Most engaged user based on the longest session time
client_id, duration =dfSessionDuration.loc[dfSessionDuration.interval.idxmax()]

print ("Most engaged user is '{}' with the session id of '{}' and session duration of '{}' ".
       format(client_id.split(sep='_')[0], client_id,duration))

Most engaged user is '106.186.23.95:35629' with the session id of '106.186.23.95:35629_1' and session duration of '0 days 18:28:30.415715' 


## Using the number of session

In [12]:
dfIpReqTime.loc[dfIpReqTime.new_session.idxmax()]

client:port                                     103.21.125.79:11676
request           GET https://paytm.com:443/papi/v1/promosearch/...
timestamp                                2015-07-22 21:08:28.711382
prev_timestamp                           2015-07-22 17:40:44.350809
interval                                     0 days 03:27:44.360573
new_session                                                       1
increment                                                         1
session_id                                    103.21.125.79:11676_1
Name: 1148093, dtype: object

## Using the number of different URLs

In [13]:
#Calculate the number of different URLs for each session

dfSummary= dfIpReqTime.groupby(['session_id'])['request'].apply(lambda x:len(list(set(x.unique().tolist()))))

dfSummary.head()


session_id
1.186.101.79:50613_0    8
1.186.101.79:50614_0    1
1.186.101.79:50895_0    6
1.186.101.79:50959_0    1
1.186.101.79:50968_0    2
Name: request, dtype: int64

In [30]:
sessionidWithGreatestURL=dfSummary.idxmax()
print("Most engaged user is '{}' with the session id of '{}' ".format(sessionidWithGreatestURL.split(sep='_')[0], sessionidWithGreatestURL))

Most engaged user is '106.51.132.54:5048' with the session id of '106.51.132.54:5048_0' 
